In [7]:
#소설 txt파일 만드는 프로그램

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import json
import time

# config.json 파일을 읽음
with open("config.json", "r",encoding='UTF8') as common:
    config = json.load(common)

chrome_options = Options()
chrome_options.add_argument("--start-maximized")
driver = webdriver.Chrome("C:/chromedriver.exe")

for genre in config["genres"]:
    # 해당 웹 사이트 이동
    driver.get(config["website_url"])
    time.sleep(3)

    # 장르 메뉴 버튼 클릭
    menu_button = driver.find_element_by_class_name("trigger-genres")
    menu_button.click()
    time.sleep(3)

    menus = driver.find_element_by_id("NAV-GENRES")
    genre_button = menus.find_element_by_link_text(genre)
    genre_button.click()
    time.sleep(3)

    # 최신 소설 선택
    novel = driver.find_element_by_css_selector("#SECTION-LIST ul li a.title")
    novel.click()
    time.sleep(3)

    # 해당 소설의 최신 에피소드 선택
    title = driver.find_element_by_css_selector("tr:not(.notice) td.subject a")
    file_name = title.text
    title.click()
    time.sleep(3)

    # 텍스트 파일로 저장
    novel_file = open(file_name + ".txt", "w+")
    lines = driver.find_elements_by_css_selector(".tcontent p")
    for line in lines:
        novel_file.write(line.text + "\n")
    novel_file.close()

driver.close()

In [137]:
import bs4
import requests

r=requests.get("https://novel.munpia.com/11799")
c=r.content
html = bs4.BeautifulSoup(c, "html.parser")

box=html.find('div',{'class','dd detail-box'})
h2=box.find('h2')
monopoly=h2.find('span')
if(monopoly==None):
    print("독점여부 : 미독점")
    title=h2.text.replace('\n',"")
    print("제목 : " + title)
    #print("제목 : " + title,end='')
else:
    print("독점여부 : " + monopoly.text)
    title=h2.text.split("독점\n")[1]
    print("제목 : " + title,end='')
    
p=box.find('p',{'class','meta-path'})
genre=p.find('strong').text
print(genre)
if ',' in genre:
    print("장르 : " + genre.split(',')[0] + ", " + genre.split(',')[1] )
else:
    print("장르 : " + genre)

days = dict.fromkeys(["mon", "tue", "wed", "thu", "fri", "sat", "sun"], False)
novel_period=box.find('div',{'class','novel-period'})
period=""
if novel_period!=None:
    if(novel_period.find('span',{'class','pluszone-mon'})):
        days.update(dict.fromkeys(["mon"], True))
        period=period+"월,"
    if(novel_period.find('span',{'class','pluszone-tue'})):
        days.update(dict.fromkeys(["tue"], True))
        period=period+"화,"
    if(novel_period.find('span',{'class','pluszone-wed'})):
        days.update(dict.fromkeys(["wed"], True))
        period=period+"수,"
    if(novel_period.find('span',{'class','pluszone-thu'})):
        days.update(dict.fromkeys(["thu"], True))
        period=period+"목,"
    if(novel_period.find('span',{'class','pluszone-fri'})):
        days.update(dict.fromkeys(["fri"], True))
        period=period+"금,"
    if(novel_period.find('span',{'class','pluszone-sat'})):
        days.update(dict.fromkeys(["sat"], True))
        period=period+"토,"
    if(novel_period.find('span',{'class','pluszone-sun'})):
        days.update(dict.fromkeys(["sun"], True))
        period=period+"일,"
    
    
    if(all(value==True for value in days.values())==True):
        period="매일"
    
    print("연재 주기 : " + period[:-1] + " 연재")

   

else:
    print("연재 주기 : 비주기적 연재")

    
age=box.find({'class','xui-icon xui-adult'})
if(age==None):
    print("연령등급 : 전체이용가")
else:
    print("연령등급 : " + age.text)

writer_dl=box.find('dl',{'class','meta-author meta'})
writer=dl.find('strong').text
print("작가 : " + writer)

etc_dl=box.findAll('dl',{'class','meta-etc meta'})
days_dl=etc_dl[0]
days_dd=days_dl.findAll('dd')
first_update=days_dd[0].text
print("작품등록일 : " + first_update)
recently_update=days_dd[1].text
print("작품등록일 : " + recently_update)

number_dl=etc_dl[1]
number_dd=number_dl.findAll('dd')

recommend=number_dd[1].text
print("추천수 : " + recommend)
enjoyer=number_dd[2].text
print("조회수 : " + enjoyer)

독점여부 : 선독점
제목 : 영웅 - 삼국지
대체역사, 판타지
장르 : 대체역사,  판타지
연재 주기 : 월,화,수,목 연재
연령등급 : 전체이용가
작가 : 와이키키진
작품등록일 : 2016.10.13 11:00
작품등록일 : 2020.04.28 23:43
추천수 : 3,826,385
조회수 : 108,882
